In [1]:
import numpy as np
from functools import partial

import sys
sys.path.append('../../')
import peach as p


import tensorly as tl
from tensorly.decomposition import parafac
from tensorly.kruskal_tensor import kruskal_to_tensor, KruskalTensor
from tensorly.base import unfold

from tensorly.decomposition import quantized_parafac
from tensorly.quantization import quantize_qint

import torch
tl.set_backend('pytorch')

In [2]:
numpy_to_struct = {np.float32 : 'f', np.int8 : 'i'}
pytorch_to_struct = {torch.float32 : 'f', torch.int8 : 'i'}

N_ITER_MAX = 10000
RANDOM_INIT_STARTS = 10


params_als_shared = {'n_iter_max' : N_ITER_MAX,\
                    'stop_criterion' : 'rec_error_decrease',\
                    'tol' : None,\
                    'normalize_factors' : True,\
                    'svd' : 'numpy_svd',\
                    'verbose' : 0}

params_als = {'orthogonalise' : False,\
              'non_negative' : False,\
              'mask' : None,\
              'return_errors' : False}

params_qint = {'dtype' : torch.qint8,\
               'qscheme' : torch.per_tensor_affine,\
               'dim' : None}

params_qals = {'qmodes' : [0, 1, 2],\
               'warmup_iters' : 1,\
               'return_scale_zeropoint' : True,\
               'return_rec_errors' : False,\
               'return_qrec_errors' : False}

In [3]:
def get_factors_init(shape, rank, dtype = 'f'):
    if dtype == 'f':
        factors = [torch.randn((i, rank)).type(torch.float32)  for i in shape] 
    elif dtype == 'i':
        factors = [torch.randint(-256, 256, (i, rank)).type(torch.int8) for i in shape]
        
    return factors

pytorch_to_struct = {torch.float32 : 'f',\
                     torch.float64 : 'd',\
                     torch.float16 : 'e',\
                     torch.int8 : 'b',\
                     torch.int16 : 'h',\
                     torch.int32 : 'i',\
                     torch.int64 : 'q',\
                     torch.uint8 : 'B',\
                     torch.qint8 : 'b',\
                     torch.quint8 : 'B',\
                     torch.qint32 : 'i'}

## Generate tensor in Kruskal format

In [94]:
rank = 12
shape = (5, 5, 5)

dtype =  torch.float32
struct_dtype = pytorch_to_struct[dtype]

factors = get_factors_init(shape, rank, dtype = struct_dtype)
weights = torch.ones(rank).type(dtype)

krt = KruskalTensor((weights, factors))
t = kruskal_to_tensor(krt)

tnorm = tl.norm(t.type(torch.float32))
print('||factors||: {}, factors type: {}'.format(tnorm, t.dtype))                                                                       


||factors||: 32.16487503051758, factors type: torch.float32


In [95]:
random_init_starts = RANDOM_INIT_STARTS
inits = ['svd'] + ['random'] * random_init_starts
random_states = [None] + [int(torch.randint(high = 11999, size = (1,))) for _ in range(random_init_starts)]


## ALS

In [96]:
rank = 11

run_id = 0
init = inits[run_id]
random_state = random_states[run_id]

out_als = parafac(t, rank,\
              init=init,\
              random_state=random_state,\
              **params_als_shared,\
              **params_als)

In [97]:
t_als = kruskal_to_tensor(out_als) 
rec_error_als = tl.norm(t - t_als)

print('||tensor - als_factors||: {}'.format(rec_error_als))

||tensor - als_factors||: 0.0024558657314628363


## ALS + post quantization

In [100]:
qfactors_post = []
scales_post = []
zero_points_post = []

for i in range(len(out_als.factors)):
    q, s, z = quantize_qint(out_als.factors[i], **params_qint, return_scale_zeropoint = True)
    qfactors_post.append(q)
    scales_post.append(s)
    zero_points_post.append(z)

qt_post = kruskal_to_tensor(KruskalTensor((out_als.weights, qfactors_post)))
rec_error_qpost = tl.norm(t - qt_post)

print('||tensor - als_factors_quantized||: {}'.format(rec_error_qpost))

||tensor - als_factors_quantized||: 1.6628392934799194


In [101]:
qfactors_post[0]/scales_post[0],  zero_points_post

(tensor([[ -11.0000,   71.0000,   66.0000,   -8.0000,   66.0000,  127.0000,
            -8.0000, -121.0000,   -8.0000,   40.0000,   48.0000],
         [  17.0000,  -12.0000,    1.0000,  -11.0000,  113.0000,   19.0000,
           -61.0000,   -6.0000,   42.0000,   42.0000,    7.0000],
         [  80.0000,  -38.0000,   -7.0000,   94.0000,   22.0000,   -1.0000,
           -21.0000,   -5.0000,    6.0000,   32.0000,  -17.0000],
         [ 108.0000,   87.0000,    3.0000,  -87.0000,   45.0000,   21.0000,
           117.0000,  -66.0000, -100.0000,  -49.0000,   16.0000],
         [  32.0000,   73.0000, -124.0000,   55.0000,   -6.0000,  -39.0000,
            42.0000,  -26.0000,   89.0000, -113.0000,  127.0000]]),
 [tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32)])

## QALS

In [112]:
(fout, qout), (errors, qerrors), scales, zero_points = quantized_parafac(t, rank,\
                                                                         init=init,\
                                                                         random_state=random_state,\
                                                                         **params_als_shared,\
                                                                         **params_qals,\
                                                                         **params_qint)

t_approx_qals = kruskal_to_tensor(KruskalTensor(qout))
rec_error_qals = tl.norm(t - t_approx_qals)

print('||tensor - qals_factors||: {}'.format(rec_error_qals))

||tensor - qals_factors||: 1.0734130144119263


In [113]:
qout.factors[0]/scales[0], qout.weights

(tensor([[-113., -128., -127.,   90.,   83.,  105.,  114.,   55., -120., -127.,
           127.],
         [  17.,  -18.,   -7.,   83.,   -9.,    5.,    3.,   30.,  -49.,  -41.,
            29.],
         [  16.,   24.,  -10.,  -29.,   39.,    1.,  -40.,  101.,   19.,  -37.,
           -29.],
         [  26.,   45.,  -55.,  -18.,  -28.,   23.,   64.,   44.,   33.,    6.,
           -30.],
         [  73.,    6.,   -2.,   57.,  101.,  -88.,   26.,  -58.,  -34.,    5.,
            25.]]),
 tensor([ 28.4415, 100.4853,  31.5890,  34.0037,   9.9575,  14.7683,  21.3021,
          22.8648,  92.4359,  42.7960,  61.0447]))

## For simulated annealing

In [114]:
def restore_factors(factors_flatten, shapes):
    factors_restored = []
    l = 0
    for shape in shapes:
        dl = np.prod(shape)
        factors_restored.append(torch.tensor(factors_flatten)[l : l + dl].reshape(shape))
        l += dl
    return factors_restored

def norm_error(factors_flatten, shapes, t, weights = None,\
               scales = None, zero_points = zero_points):
    
    factors_restored = restore_factors(factors_flatten, shapes)
    if scales is not None:
        factors_restored = [(factors_restored[i] - zero_points[0])*scales[i] for i in range(len(scales))]
        
    tensor_restored = kruskal_to_tensor(KruskalTensor((weights, factors_restored)))

    tensor_restored.shape

    norm_error = tl.norm(tensor_restored - t)
    
    print('Norm error: {}'.format(norm_error))
    
    return float(norm_error)

## ALS + post quantization + simulated annealing

In [105]:
qfactors_post[0]/scales_post[0], scales_post, zero_points, out_als.weights

(tensor([[ -11.0000,   71.0000,   66.0000,   -8.0000,   66.0000,  127.0000,
            -8.0000, -121.0000,   -8.0000,   40.0000,   48.0000],
         [  17.0000,  -12.0000,    1.0000,  -11.0000,  113.0000,   19.0000,
           -61.0000,   -6.0000,   42.0000,   42.0000,    7.0000],
         [  80.0000,  -38.0000,   -7.0000,   94.0000,   22.0000,   -1.0000,
           -21.0000,   -5.0000,    6.0000,   32.0000,  -17.0000],
         [ 108.0000,   87.0000,    3.0000,  -87.0000,   45.0000,   21.0000,
           117.0000,  -66.0000, -100.0000,  -49.0000,   16.0000],
         [  32.0000,   73.0000, -124.0000,   55.0000,   -6.0000,  -39.0000,
            42.0000,  -26.0000,   89.0000, -113.0000,  127.0000]]),
 [tensor(0.0071), tensor(0.0068), tensor(0.0064)],
 (tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32)),
 tensor([18.2783, 13.1356, 31.3312, 10.5132,  7.3779, 27.4542, 22.1909, 27.4600,
         20.4559, 19.8272, 20.6246]))

In [147]:
qfactors_post_int = [qfactors_post[i]/scales_post[i]\
                     for i in range(len(qfactors_post))]

shapes = [factor.shape for factor in qfactors_post_int]
factors_flatten_init =  torch.cat([factor.flatten() for factor in qfactors_post_int])
factors_flatten_init = factors_flatten_init.type(torch.int8)


bsa = None

# numpy_to_struct = {np.float32 : 'f', np.float64 : 'd',  np.float16 : 'e',\
#                    np.int8 : 'b', np.int16 : 'h', np.int32 : 'i', np.int64 : 'q',\
#                    np.uint8 : 'B', np.uint16 : 'H', np.uint32 : 'I', np.uint64 : 'Q'}


mask = np.zeros(8).astype(bool)
mask[-2:] = True

neighbor_params = {'nb':2, 'mask': mask}
neighbor = p.sa.MaskedBitsNeighbor(**neighbor_params)

bsa = p.BinarySA(partial(norm_error,\
                         shapes = shapes, t = t, weights = out_als.weights,\
                         scales = scales_post, zero_points = zero_points_post),\
                 np.array(factors_flatten_init).astype(np.int8),\
#                  np.array(flatten_factors_opt),\
                 [(-2**7, 2**7)]*len(factors_flatten_init),\
                 'i'*len(factors_flatten_init),\
                  emax = 1e-10,\
                  imax = 10000,\
                  T0 = 1,\
                  rt = 0.8, neighbor = neighbor)

Norm error: 1.7254672050476074


In [148]:
# make sense to run several times
for _ in range(10):
    flatten_factors_opt, error =  bsa()

Norm error: 1.7254672050476074
Norm error: 16.001171112060547
Norm error: 1.7254672050476074
Norm error: 3.0041446685791016
Norm error: 1.7254672050476074
Norm error: 4.930253028869629
Norm error: 1.7254672050476074
Norm error: 3.5791451930999756
Norm error: 1.7254672050476074
Norm error: 1.737926959991455
Norm error: 1.737926959991455
Norm error: 5.399327754974365
Norm error: 1.737926959991455
Norm error: 5.676782131195068
Norm error: 1.737926959991455
Norm error: 26.551612854003906
Norm error: 1.737926959991455
Norm error: 14.228104591369629
Norm error: 1.737926959991455
Norm error: 22.736330032348633
Norm error: 1.737926959991455
Norm error: 18.183929443359375
Norm error: 1.737926959991455
Norm error: 23.667015075683594
Norm error: 1.737926959991455
Norm error: 9.285061836242676
Norm error: 1.737926959991455
Norm error: 9.907973289489746
Norm error: 1.737926959991455
Norm error: 26.034509658813477
Norm error: 1.737926959991455
Norm error: 13.632533073425293
Norm error: 1.73792695999

Norm error: 9.309471130371094
Norm error: 1.636426568031311
Norm error: 33.81692886352539
Norm error: 1.636426568031311
Norm error: 3.8807928562164307
Norm error: 1.636426568031311
Norm error: 13.664502143859863
Norm error: 1.636426568031311
Norm error: 53.67015838623047
Norm error: 1.636426568031311
Norm error: 20.287084579467773
Norm error: 1.636426568031311
Norm error: 18.42121696472168
Norm error: 1.636426568031311
Norm error: 2.277266263961792
Norm error: 1.636426568031311
Norm error: 17.51618194580078
Norm error: 1.636426568031311
Norm error: 14.228089332580566
Norm error: 1.636426568031311
Norm error: 42.012168884277344
Norm error: 1.636426568031311
Norm error: 8.88137149810791
Norm error: 1.636426568031311
Norm error: 8.618362426757812
Norm error: 1.636426568031311
Norm error: 23.319067001342773
Norm error: 1.636426568031311
Norm error: 7.058661937713623
Norm error: 1.636426568031311
Norm error: 20.96738052368164
Norm error: 1.636426568031311
Norm error: 20.828128814697266
Norm

Norm error: 2.8557801246643066
Norm error: 1.563374638557434
Norm error: 2.5675911903381348
Norm error: 1.563374638557434
Norm error: 20.905494689941406
Norm error: 1.563374638557434
Norm error: 16.966873168945312
Norm error: 1.563374638557434
Norm error: 18.849109649658203
Norm error: 1.563374638557434
Norm error: 10.275421142578125
Norm error: 1.563374638557434
Norm error: 36.880889892578125
Norm error: 1.563374638557434
Norm error: 13.366390228271484
Norm error: 1.563374638557434
Norm error: 6.996446132659912
Norm error: 1.563374638557434
Norm error: 17.815359115600586
Norm error: 1.563374638557434
Norm error: 17.499893188476562
Norm error: 1.563374638557434
Norm error: 9.334702491760254
Norm error: 1.563374638557434
Norm error: 14.359713554382324
Norm error: 1.563374638557434
Norm error: 13.283616065979004
Norm error: 1.563374638557434
Norm error: 31.332080841064453
Norm error: 1.563374638557434
Norm error: 9.93116569519043
Norm error: 1.563374638557434
Norm error: 34.7456321716308

Norm error: 31.60723876953125
Norm error: 1.224191427230835
Norm error: 6.793470859527588
Norm error: 1.224191427230835
Norm error: 32.341407775878906
Norm error: 1.224191427230835
Norm error: 8.555708885192871
Norm error: 1.224191427230835
Norm error: 2.2896106243133545
Norm error: 1.224191427230835
Norm error: 1.5478686094284058
Norm error: 1.224191427230835
Norm error: 16.283966064453125
Norm error: 1.224191427230835
Norm error: 23.693798065185547
Norm error: 1.224191427230835
Norm error: 1.9303696155548096
Norm error: 1.224191427230835
Norm error: 8.44824504852295
Norm error: 1.224191427230835
Norm error: 32.14620590209961
Norm error: 1.224191427230835
Norm error: 27.461669921875
Norm error: 1.224191427230835
Norm error: 19.549659729003906
Norm error: 1.224191427230835
Norm error: 5.107477188110352
Norm error: 1.224191427230835
Norm error: 41.709320068359375
Norm error: 1.224191427230835
Norm error: 11.273181915283203
Norm error: 1.224191427230835
Norm error: 26.24722671508789
Norm

Norm error: 19.28007698059082
Norm error: 1.1938709020614624
Norm error: 7.692564010620117
Norm error: 1.1938709020614624
Norm error: 23.940061569213867
Norm error: 1.1938709020614624
Norm error: 11.779748916625977
Norm error: 1.1938709020614624
Norm error: 6.072939872741699
Norm error: 1.1938709020614624
Norm error: 37.89494323730469
Norm error: 1.1938709020614624
Norm error: 9.702713966369629
Norm error: 1.1938709020614624
Norm error: 5.4652485847473145
Norm error: 1.1938709020614624
Norm error: 9.657567024230957
Norm error: 1.1938709020614624
Norm error: 19.36121368408203
Norm error: 1.1938709020614624
Norm error: 18.735858917236328
Norm error: 1.1938709020614624
Norm error: 27.215410232543945
Norm error: 1.1938709020614624
Norm error: 18.53234100341797
Norm error: 1.1938709020614624
Norm error: 28.661304473876953
Norm error: 1.1938709020614624
Norm error: 22.318666458129883
Norm error: 1.1938709020614624
Norm error: 6.240851879119873
Norm error: 1.1938709020614624
Norm error: 20.32

...

Norm error: 29.452524185180664
Norm error: 0.6101324558258057
Norm error: 0.6876997947692871
Norm error: 0.6101324558258057
Norm error: 1.786077857017517
Norm error: 0.6101324558258057
Norm error: 23.31219482421875
Norm error: 0.6101324558258057
Norm error: 14.124359130859375
Norm error: 0.6101324558258057
Norm error: 13.33823299407959
Norm error: 0.6101324558258057
Norm error: 28.97051429748535
Norm error: 0.6101324558258057
Norm error: 0.6781541109085083
Norm error: 0.6101324558258057
Norm error: 6.5373334884643555
Norm error: 0.6101324558258057
Norm error: 19.342653274536133
Norm error: 0.6101324558258057
Norm error: 19.85118293762207
Norm error: 0.6101324558258057
Norm error: 17.49095916748047
Norm error: 0.6101324558258057
Norm error: 0.6850709319114685
Norm error: 0.6101324558258057
Norm error: 15.574746131896973
Norm error: 0.6101324558258057
Norm error: 21.597841262817383
Norm error: 0.6101324558258057
Norm error: 21.990692138671875
Norm error: 0.6101324558258057
Norm error: 24

Norm error: 10.556479454040527
Norm error: 0.6101324558258057
Norm error: 39.445709228515625
Norm error: 0.6101324558258057
Norm error: 19.025243759155273
Norm error: 0.6101324558258057
Norm error: 34.70122146606445
Norm error: 0.6101324558258057
Norm error: 6.782497882843018
Norm error: 0.6101324558258057
Norm error: 14.154960632324219
Norm error: 0.6101324558258057
Norm error: 9.746748924255371
Norm error: 0.6101324558258057
Norm error: 12.515213966369629
Norm error: 0.6101324558258057
Norm error: 13.320968627929688
Norm error: 0.6101324558258057
Norm error: 9.68126106262207
Norm error: 0.6101324558258057
Norm error: 29.54877471923828
Norm error: 0.6101324558258057
Norm error: 26.3845157623291
Norm error: 0.6101324558258057
Norm error: 7.774478912353516
Norm error: 0.6101324558258057
Norm error: 14.220892906188965
Norm error: 0.6101324558258057
Norm error: 0.6702919006347656
Norm error: 0.6101324558258057
Norm error: 28.76048469543457
Norm error: 0.6101324558258057
Norm error: 8.5659

Norm error: 28.586645126342773
Norm error: 0.6101324558258057
Norm error: 23.267505645751953
Norm error: 0.6101324558258057
Norm error: 5.212019443511963
Norm error: 0.6101324558258057
Norm error: 27.60045051574707
Norm error: 0.6101324558258057
Norm error: 9.118359565734863
Norm error: 0.6101324558258057
Norm error: 17.159324645996094
Norm error: 0.6101324558258057
Norm error: 30.785484313964844
Norm error: 0.6101324558258057
Norm error: 21.833242416381836
Norm error: 0.6101324558258057
Norm error: 23.410118103027344
Norm error: 0.6101324558258057
Norm error: 0.6267121434211731
Norm error: 0.6101324558258057
Norm error: 11.221315383911133
Norm error: 0.6101324558258057
Norm error: 4.288424968719482
Norm error: 0.6101324558258057
Norm error: 8.362183570861816
Norm error: 0.6101324558258057
Norm error: 14.185409545898438
Norm error: 0.6101324558258057
Norm error: 24.873422622680664
Norm error: 0.6101324558258057
Norm error: 17.646432876586914
Norm error: 0.6101324558258057
Norm error: 1

Norm error: 6.601365566253662
Norm error: 0.597341001033783
Norm error: 24.844358444213867
Norm error: 0.597341001033783
Norm error: 27.071796417236328
Norm error: 0.597341001033783
Norm error: 3.4888534545898438
Norm error: 0.597341001033783
Norm error: 14.674839973449707
Norm error: 0.597341001033783
Norm error: 19.145599365234375
Norm error: 0.597341001033783
Norm error: 21.512048721313477
Norm error: 0.597341001033783
Norm error: 3.61232590675354
Norm error: 0.597341001033783
Norm error: 39.92742919921875
Norm error: 0.597341001033783
Norm error: 8.956063270568848
Norm error: 0.597341001033783
Norm error: 15.394867897033691
Norm error: 0.597341001033783
Norm error: 8.837478637695312
Norm error: 0.597341001033783
Norm error: 30.196857452392578
Norm error: 0.597341001033783
Norm error: 12.639484405517578
Norm error: 0.597341001033783
Norm error: 10.69424057006836
Norm error: 0.597341001033783
Norm error: 12.876679420471191
Norm error: 0.597341001033783
Norm error: 17.00538444519043
N

Norm error: 9.164861679077148
Norm error: 0.597341001033783
Norm error: 32.05413055419922
Norm error: 0.597341001033783
Norm error: 4.3265814781188965
Norm error: 0.597341001033783
Norm error: 20.317689895629883
Norm error: 0.597341001033783
Norm error: 12.74379825592041
Norm error: 0.597341001033783
Norm error: 3.3213512897491455
Norm error: 0.597341001033783
Norm error: 22.531536102294922
Norm error: 0.597341001033783
Norm error: 26.17209243774414
Norm error: 0.597341001033783
Norm error: 30.633987426757812
Norm error: 0.597341001033783
Norm error: 1.109581708908081
Norm error: 0.597341001033783
Norm error: 23.962326049804688
Norm error: 0.597341001033783
Norm error: 8.492817878723145
Norm error: 0.597341001033783
Norm error: 12.491734504699707
Norm error: 0.597341001033783
Norm error: 2.0719690322875977
Norm error: 0.597341001033783
Norm error: 9.578672409057617
Norm error: 0.597341001033783
Norm error: 8.742929458618164
Norm error: 0.597341001033783
Norm error: 13.021952629089355
N

## QALS + simulated annealing

In [115]:
qout.factors[0]/scales[0], scales, zero_points, qout.weights

(tensor([[-113., -128., -127.,   90.,   83.,  105.,  114.,   55., -120., -127.,
           127.],
         [  17.,  -18.,   -7.,   83.,   -9.,    5.,    3.,   30.,  -49.,  -41.,
            29.],
         [  16.,   24.,  -10.,  -29.,   39.,    1.,  -40.,  101.,   19.,  -37.,
           -29.],
         [  26.,   45.,  -55.,  -18.,  -28.,   23.,   64.,   44.,   33.,    6.,
           -30.],
         [  73.,    6.,   -2.,   57.,  101.,  -88.,   26.,  -58.,  -34.,    5.,
            25.]]),
 (tensor(0.0072), tensor(0.0073), tensor(0.0059)),
 (tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32),
  tensor(0, dtype=torch.int32)),
 tensor([ 28.4415, 100.4853,  31.5890,  34.0037,   9.9575,  14.7683,  21.3021,
          22.8648,  92.4359,  42.7960,  61.0447]))

In [159]:
qout_factors_int = [qout.factors[i]/scales[i] for i in range(len(qout.factors))]

shapes = [factor.shape for factor in qout_factors_int]
factors_flatten_init =  torch.cat([factor.flatten() for factor in qout_factors_int])
factors_flatten_init = factors_flatten_init.type(torch.int8)


bsa = None

# numpy_to_struct = {np.float32 : 'f', np.float64 : 'd',  np.float16 : 'e',\
#                    np.int8 : 'b', np.int16 : 'h', np.int32 : 'i', np.int64 : 'q',\
#                    np.uint8 : 'B', np.uint16 : 'H', np.uint32 : 'I', np.uint64 : 'Q'}


mask = np.zeros(8).astype(bool)
mask[-2:] = True

neighbor_params = {'nb':2, 'mask': mask}
neighbor = p.sa.MaskedBitsNeighbor(**neighbor_params)

bsa = p.BinarySA(partial(norm_error,\
                         shapes = shapes, t = t, weights = qout.weights,\
                         scales = scales, zero_points = zero_points),\
                 np.array(factors_flatten_init).astype(np.int8),\
#                  np.array(flatten_factors_opt),\
                 [(-2**7, 2**7)]*len(factors_flatten_init),\
                 'i'*len(factors_flatten_init),\
                  emax = 1e-10,\
                  imax = 5000,\
                  T0 = 1,\
                  rt = 0.8, neighbor = neighbor)

Norm error: 1.0471503734588623


In [162]:
# make sense to run several times
for j in range(1):
    print('=========================={}=========================='.format(j))
    flatten_factors_opt, error =  bsa()

==========================0==========================
Norm error: 0.6777209639549255
Norm error: 92.2162857055664
Norm error: 0.6777209639549255
Norm error: 75.33322143554688
Norm error: 0.6777209639549255
Norm error: 0.952441394329071
Norm error: 0.6777209639549255
Norm error: 2.2045059204101562
Norm error: 0.6777209639549255
Norm error: 80.71922302246094
Norm error: 0.6777209639549255
Norm error: 5.302135467529297
Norm error: 0.6777209639549255
Norm error: 10.32032585144043
Norm error: 0.6777209639549255
Norm error: 16.556419372558594
Norm error: 0.6777209639549255
Norm error: 68.00707244873047
Norm error: 0.6777209639549255
Norm error: 26.313213348388672
Norm error: 0.6777209639549255
Norm error: 10.988495826721191
Norm error: 0.6777209639549255
Norm error: 7.755836009979248
Norm error: 0.6777209639549255
Norm error: 0.9933557510375977
Norm error: 0.6777209639549255
Norm error: 0.8396408557891846
Norm error: 0.6777209639549255
Norm error: 6.823709011077881
Norm error: 0.677720963954

In [163]:
flatten_factors_opt

(-114,
 -128,
 -127,
 90,
 81,
 105,
 115,
 55,
 -120,
 -127,
 126,
 17,
 -18,
 -7,
 83,
 -9,
 5,
 3,
 29,
 -49,
 -41,
 29,
 16,
 24,
 -9,
 -29,
 39,
 1,
 -40,
 101,
 19,
 -37,
 -29,
 26,
 45,
 -55,
 -18,
 -27,
 22,
 64,
 44,
 33,
 6,
 -30,
 73,
 6,
 -2,
 57,
 101,
 -88,
 27,
 -57,
 -34,
 5,
 25,
 26,
 -10,
 22,
 17,
 -70,
 -78,
 -11,
 3,
 -14,
 7,
 11,
 29,
 4,
 10,
 37,
 69,
 -23,
 58,
 97,
 -43,
 105,
 9,
 -56,
 -37,
 92,
 7,
 22,
 89,
 -13,
 34,
 -1,
 25,
 71,
 -100,
 -97,
 82,
 126,
 -75,
 35,
 -100,
 91,
 -128,
 80,
 45,
 -66,
 -89,
 57,
 36,
 -58,
 -57,
 -73,
 11,
 -17,
 -26,
 108,
 -5,
 -36,
 -73,
 -24,
 74,
 -51,
 72,
 81,
 49,
 79,
 40,
 69,
 -27,
 -99,
 80,
 -74,
 -128,
 123,
 26,
 -13,
 9,
 -8,
 56,
 42,
 86,
 65,
 -83,
 17,
 80,
 -40,
 -37,
 -28,
 -12,
 126,
 -125,
 45,
 -90,
 -5,
 -54,
 -2,
 82,
 105,
 84,
 123,
 67,
 -96,
 62,
 -96,
 104,
 78,
 -43,
 114,
 117,
 120,
 108)